# Coastal change in African countries

* **Products used:** [DE Africa Coastlines]()



## Background

Coastline change can be summarised in country scale to assess trends of change. 

## Description

Coastlines data is downloaded and used for analysis so its not impacted by limited number of features returned by WFS.

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

### Load packages
Import Python packages that are used for the analysis.

In [ ]:
import geopandas as gpd
import rioxarray as rxr
import numpy as np
from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.coastal import get_coastlines
#from deafrica_tools.plotting import display_map
from datacube.utils.geometry import Geometry

In [ ]:
# set s3 region for HRSL data access
import os
os.environ['AWS_DEFAULT_REGION']="us-east-1"
os.environ['AWS_S3_ENDPOINT']="s3.us-east-1.amazonaws.com"


In [ ]:
import fiona
fiona.listlayers("../../data/deafricacoastlines_v0.4.0.gpkg")

In [ ]:
%%time
coastlines = gpd.read_file("../../../data/deafricacoastlines_v0.4.0.gpkg", layer="rates_of_change")

In [ ]:
african_countries  = gpd.read_file("../Supplementary_data/MGCI/african_countries.geojson")

In [ ]:
african_countries[['n_good', 'n_sig99', 'n_neg', 'n_neg1', 'n_neg5', 
                   'rate_time_10th', 'rate_time_50th', 'nsm_10th', 'sce_90th', 
                   'pop_neg1', 'pop_neg5']] = [np.nan]*11

In [ ]:
african_countries["COUNTRY_ALT"]=african_countries["COUNTRY"]

In [ ]:
african_countries["COUNTRY"].values

In [ ]:
alt_name = {'Comoros':'Comores','Congo-Brazzaville':'Republic of the Congo', 'Cote d`Ivoire':'Ivory Coast', 
            'Democratic Republic of Congo':'Democratic Republic of the Congo','Somalia':'Federal Republic of Somalia'}
for key, value in alt_name.items():
    print(key, value, african_countries[african_countries["COUNTRY"]==key].index[0])
    african_countries.at[african_countries[african_countries["COUNTRY"]==key].index[0], "COUNTRY_ALT"]=value

In [ ]:
np.sort(coastlines[coastlines.country.str.len()>0].country.unique())

In [ ]:
for idx in african_countries.index:
    print(african_countries.loc[idx].COUNTRY)
    change_gdf=coastlines[coastlines.country==african_countries.loc[idx].COUNTRY_ALT]
    #country_geom = Geometry(geom=african_countries.loc[idx].geometry, crs=african_countries.crs)
    #bbox = list(country_geom.boundingbox)
    #change_gdf = get_coastlines(bbox=bbox, layer='statistics')
    if len(change_gdf)==0:
        print("no data over", african_countries.loc[idx].COUNTRY)
        continue

    # get stats
    good_gdf = change_gdf[(change_gdf.certainty=='good')]
    african_countries.at[idx, 'n_good'] = len(good_gdf)
    sig_gdf = good_gdf[(good_gdf.sig_time<0.01)]
    african_countries.at[idx, 'n_sig99'] = len(sig_gdf)
    african_countries.at[idx, 'n_neg'] = (sig_gdf.rate_time<0).sum()
    african_countries.at[idx, 'n_neg1'] = (sig_gdf.rate_time<-1).sum()
    african_countries.at[idx, 'n_neg5'] = (sig_gdf.rate_time<-5).sum()
    african_countries.at[idx, 'rate_time_10th'] = sig_gdf.rate_time.quantile(0.1)
    african_countries.at[idx, 'rate_time_50th'] = sig_gdf.rate_time.quantile(0.5)
    african_countries.at[idx, 'nsm_10th'] = sig_gdf.nsm.quantile(0.1)
    african_countries.at[idx, 'sce_90th'] = sig_gdf.sce.quantile(0.9)

    # get hotspots and population
    # Load population data
    ds = rxr.open_rasterio('s3://dataforgood-fb-data/hrsl-cogs/hrsl_general/hrsl_general-latest.vrt', 
                       chunks ={'x': 1000, 'y': 1000})
    ds=ds.rename({'x':'longitude','y':'latitude'})

    # identify hotspots
    buffer=20
    buffered = change_gdf[(change_gdf.certainty=='good') & (change_gdf.sig_time<0.01) & (change_gdf.rate_time<-1)].copy()
    if len(buffered)==0:
        african_countries.at[idx, 'pop_neg1'] = 0 
        african_countries.at[idx, 'pop_neg5'] = 0
        continue
    buffered['geometry'] = buffered.to_crs('6933').buffer(buffer).to_crs('4326')
    hotspots = gpd.overlay(buffered, buffered, how='intersection')
    coast_bbox = buffered.unary_union.bounds
    pop_raster = ds.isel(band=0).sel(longitude=slice(coast_bbox[0],coast_bbox[2]), latitude = slice(coast_bbox[3], coast_bbox[1]))
    hotspots_mask = xr_rasterize(gdf=hotspots,
                                  da=pop_raster,
                                  transform=pop_raster.geobox.transform,
                                  crs=pop_raster.geobox.crs)
    african_countries.at[idx, 'pop_neg1'] = (pop_raster*hotspots_mask).sum().values
    
    buffered = change_gdf[(change_gdf.certainty=='good') & (change_gdf.sig_time<0.01) & (change_gdf.rate_time<-5)].copy()
    buffered['geometry'] = buffered.to_crs('6933').buffer(buffer).to_crs('4326')
    hotspots = gpd.overlay(buffered, buffered, how='intersection')
    coast_bbox = buffered.unary_union.bounds
    pop_raster = ds.isel(band=0).sel(longitude=slice(coast_bbox[0],coast_bbox[2]), latitude = slice(coast_bbox[3], coast_bbox[1]))
    hotspots_mask = xr_rasterize(gdf=hotspots,
                                  da=pop_raster,
                                  transform=pop_raster.geobox.transform,
                                  crs=pop_raster.geobox.crs)
    african_countries.at[idx, 'pop_neg5'] = (pop_raster*hotspots_mask).sum().values

In [ ]:
african_countries['perc_neg']=african_countries['n_neg']*100/african_countries['n_good']
african_countries['perc_neg1']=african_countries['n_neg1']*100/african_countries['n_good']
african_countries['perc_neg5']=african_countries['n_neg5']*100/african_countries['n_good']

african_countries['km_neg']=african_countries['n_neg']*30/1000
african_countries['km_neg1']=african_countries['n_neg1']*30/1000
african_countries['km_neg5']=african_countries['n_neg5']*30/1000

In [ ]:
#african_countries = african_countries[african_countries.n_good>0]

In [ ]:
african_countries.to_file("african_countries_coastalchange.geojson")

In [ ]:
african_countries_coastalchange  = gpd.read_file("african_countries_coastalchange.geojson")

In [ ]:
from matplotlib import pyplot as plt
import matplotlib as mpl

param = 'perc_neg1'
title = 'Percentage of shorelines retreated at more than 1 meter per year'

steps = np.ceil((african_countries_coastalchange[param].max()/6)/10)*10
vmin, vmax = 0, np.ceil(african_countries_coastalchange[param].max()/steps)*steps+steps

# Define plot and colourbar axes
fig, ax = plt.subplots(1,1, figsize=(10,10))
fig.subplots_adjust(bottom=0.2)
cax = fig.add_axes([0.16, 0.15, 0.70, 0.03])


# Define colour map
cmap = mpl.cm.YlOrRd
bounds = list(np.arange(vmin,vmax,steps))
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)#, extend='both')
cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap,
                                norm=norm,
                                orientation='horizontal')

# Define colourbar labelling
# cbar.set_ticks([])
cbar.set_ticks([i for i in np.arange(vmin,vmax,steps)])
cbar.set_ticklabels(list('{:.0f}'.format(i) for i in (list(np.arange(vmin, vmax, steps)))))
cbar.set_label(title, fontsize='14')

# Turn off lon-lat ticks and labels
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_yticks([])

# # Remove frame
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

african_countries_coastalchange.plot(param, ax=ax, cmap=cmap, norm=norm);
african_countries_coastalchange.geometry.plot(ax=ax, linewidth=0.8, edgecolor='black', facecolor="none");


In [ ]:
param = 'km_neg1'
title = 'Length (km) of shorelines retreated at more than 1 meter per year'

steps = np.ceil((african_countries_coastalchange[param].max()/10)/100)*100
vmin, vmax = 0, np.ceil(african_countries_coastalchange[param].max()/steps)*steps+steps

# Define plot and colourbar axes
fig, ax = plt.subplots(1,1, figsize=(10,10))
fig.subplots_adjust(bottom=0.2)
cax = fig.add_axes([0.16, 0.15, 0.70, 0.03])


# Define colour map
cmap = mpl.cm.YlOrRd
bounds = list(np.arange(vmin,vmax,steps))
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)#, extend='both')
cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap,
                                norm=norm,
                                orientation='horizontal')

# Define colourbar labelling
# cbar.set_ticks([])
cbar.set_ticks([i for i in np.arange(vmin,vmax,steps)])
cbar.set_ticklabels(list('{:.0f}'.format(i) for i in (list(np.arange(vmin, vmax, steps)))))
cbar.set_label(title, fontsize='14')

# Turn off lon-lat ticks and labels
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_yticks([])

# # Remove frame
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

african_countries_coastalchange.plot(param, ax=ax, cmap=cmap, norm=norm);
african_countries_coastalchange.geometry.plot(ax=ax, linewidth=0.8, edgecolor='black', facecolor="none");


In [ ]:
param = 'rate_time_10th'
title = '10th percentile rate of change (m per year)'

steps = np.ceil(abs(african_countries_coastalchange[param].min())/10)
vmin, vmax = np.floor(african_countries_coastalchange[param].min()/steps)*steps+steps, 0 

# Define plot and colourbar axes
fig, ax = plt.subplots(1,1, figsize=(10,10))
fig.subplots_adjust(bottom=0.2)
cax = fig.add_axes([0.16, 0.15, 0.70, 0.03])


# Define colour map
cmap = mpl.cm.YlOrRd_r
bounds = list(np.arange(vmin,vmax,steps))
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)#, extend='both')
cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap,
                                norm=norm,
                                orientation='horizontal')

# Define colourbar labelling
# cbar.set_ticks([])
cbar.set_ticks([i for i in np.arange(vmin,vmax,steps)])
cbar.set_ticklabels(list('{:.0f}'.format(i) for i in (list(np.arange(vmin, vmax, steps)))))
cbar.set_label(title, fontsize='14')

# Turn off lon-lat ticks and labels
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_yticks([])

# # Remove frame
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

african_countries_coastalchange.plot(param, ax=ax, cmap=cmap, norm=norm);
african_countries_coastalchange.geometry.plot(ax=ax, linewidth=0.8, edgecolor='black', facecolor="none");


In [ ]:
param = 'nsm_10th'
title = '10th percentile Net Shoreline Movement (metre)'

steps = np.floor(abs(african_countries_coastalchange[param].min())/10/10)*10
vmin, vmax = np.floor(african_countries_coastalchange[param].min()/steps)*steps+steps, 0 

# Define plot and colourbar axes
fig, ax = plt.subplots(1,1, figsize=(10,10))
fig.subplots_adjust(bottom=0.2)
cax = fig.add_axes([0.16, 0.15, 0.70, 0.03])


# Define colour map
cmap = mpl.cm.YlOrRd_r
bounds = list(np.arange(vmin,vmax,steps))
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)#, extend='both')
cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap,
                                norm=norm,
                                orientation='horizontal')

# Define colourbar labelling
# cbar.set_ticks([])
cbar.set_ticks([i for i in np.arange(vmin,vmax,steps)])
cbar.set_ticklabels(list('{:.0f}'.format(i) for i in (list(np.arange(vmin, vmax, steps)))))
cbar.set_label(title, fontsize='14')

# Turn off lon-lat ticks and labels
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_yticks([])

# # Remove frame
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

african_countries_coastalchange.plot(param, ax=ax, cmap=cmap, norm=norm);
african_countries_coastalchange.geometry.plot(ax=ax, linewidth=0.8, edgecolor='black', facecolor="none");

In [ ]:
param = 'sce_90th'
title = '90th percentile Shoreline Change Envelope (metre)'

steps = np.ceil((african_countries_coastalchange[param].max()/6)/10)*10
vmin, vmax = 0, np.ceil(african_countries_coastalchange[param].max()/steps)*steps+steps


# Define plot and colourbar axes
fig, ax = plt.subplots(1,1, figsize=(10,10))
fig.subplots_adjust(bottom=0.2)
cax = fig.add_axes([0.16, 0.15, 0.70, 0.03])


# Define colour map
cmap = mpl.cm.YlOrRd
bounds = list(np.arange(vmin,vmax,steps))
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)#, extend='both')
cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap,
                                norm=norm,
                                orientation='horizontal')

# Define colourbar labelling
# cbar.set_ticks([])
cbar.set_ticks([i for i in np.arange(vmin,vmax,steps)])
cbar.set_ticklabels(list('{:.0f}'.format(i) for i in (list(np.arange(vmin, vmax, steps)))))
cbar.set_label(title, fontsize='14')

# Turn off lon-lat ticks and labels
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_yticks([])

# # Remove frame
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

african_countries_coastalchange.plot(param, ax=ax, cmap=cmap, norm=norm);
african_countries_coastalchange.geometry.plot(ax=ax, linewidth=0.8, edgecolor='black', facecolor="none");

In [ ]:
param = 'pop_neg1'
title = 'Total population near shoreline retreating more than 1 m per year'

steps = np.ceil((african_countries_coastalchange[param].max()/6)/1000)*1000
vmin, vmax = 0, np.ceil(african_countries_coastalchange[param].max()/steps)*steps+steps

# Define plot and colourbar axes
fig, ax = plt.subplots(1,1, figsize=(10,10))
fig.subplots_adjust(bottom=0.2)
cax = fig.add_axes([0.16, 0.15, 0.70, 0.03])


# Define colour map
cmap = mpl.cm.YlOrRd
bounds = list(np.arange(vmin,vmax,steps))
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)#, extend='both')
cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap,
                                norm=norm,
                                orientation='horizontal')

# Define colourbar labelling
# cbar.set_ticks([])
cbar.set_ticks([i for i in np.arange(vmin,vmax,steps)])
cbar.set_ticklabels(list('{:.0f}'.format(i) for i in (list(np.arange(vmin, vmax, steps)))))
cbar.set_label(title, fontsize='14')

# Turn off lon-lat ticks and labels
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_yticks([])

# # Remove frame
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

african_countries_coastalchange.plot(param, ax=ax, cmap=cmap, norm=norm);
african_countries_coastalchange.geometry.plot(ax=ax, linewidth=0.8, edgecolor='black', facecolor="none");


In [ ]:
param = 'pop_neg5'
title = 'Total population near shoreline retreating more than 5 m per year'

steps = np.ceil((african_countries_coastalchange[param].max()/6)/1000)*1000
vmin, vmax = 0, np.ceil(african_countries_coastalchange[param].max()/steps)*steps+steps

# Define plot and colourbar axes
fig, ax = plt.subplots(1,1, figsize=(10,10))
fig.subplots_adjust(bottom=0.2)
cax = fig.add_axes([0.16, 0.15, 0.70, 0.03])


# Define colour map
cmap = mpl.cm.YlOrRd
bounds = list(np.arange(vmin,vmax,steps))
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)#, extend='both')
cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap,
                                norm=norm,
                                orientation='horizontal')

# Define colourbar labelling
# cbar.set_ticks([])
cbar.set_ticks([i for i in np.arange(vmin,vmax,steps)])
cbar.set_ticklabels(list('{:.0f}'.format(i) for i in (list(np.arange(vmin, vmax, steps)))))
cbar.set_label(title, fontsize='14')

# Turn off lon-lat ticks and labels
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_yticks([])

# # Remove frame
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

african_countries_coastalchange.plot(param, ax=ax, cmap=cmap, norm=norm);
african_countries_coastalchange.geometry.plot(ax=ax, linewidth=0.8, edgecolor='black', facecolor="none");
